# Predicción de Precios de Acciones Usando Modelos de Machine Learning

En este notebook se explora cómo construir y evaluar modelos de regresión para predecir el precio futuro de acciones a partir de datos históricos del mercado financiero. Se utilizan técnicas de aprendizaje automático para analizar variables como apertura, cierre, volumen y retornos diarios, permitiendo estimar el comportamiento futuro de los precios y apoyar la toma de decisiones en inversiones.

In [2]:
### 

def plot_stock_data(data, colors):
    plt.style.use('dark_background')
    fig, axs = plt.subplots(3, 1, figsize=(12, 8))
    
    # Configuration
    plot_configs = [
      {'data_col': 'Daily_Return', 'ax': axs[0], 'label': 'Daily Return', 'title': 'Daily Returns', 'ylabel': 'Daily Return', 'color': colors['daily_return']},
      {'data_col': 'Open', 'ax': axs[1], 'label': 'Open', 'title': 'Open, High, and Low Prices', 'ylabel': 'Price (USD)', 'color': colors['open']},
      {'data_col': 'High', 'ax': axs[1], 'label': 'High', 'color': colors['high']},
      {'data_col': 'Low', 'ax': axs[1], 'label': 'Low', 'color': colors['low']},
      {'data_col': 'Volume', 'ax': axs[2], 'label': 'Volume', 'title': 'Trading Volume', 'ylabel': 'Volume', 'color': colors['volume'], 'bar': True},
    ]


    for config in plot_configs:
        if 'bar' in config:
            config['ax'].bar(data.index, data[config['data_col']], label=config['label'], color=config['color'])
        else:
            config['ax'].plot(data.index, data[config['data_col']], label=config['label'], color=config['color'])
        
        if 'title' in config:
            config['ax'].set_title(config['title'])
        if 'ylabel' in config:
            config['ax'].set_ylabel(config['ylabel'])
        config['ax'].legend()
    
    plt.tight_layout()
```

<p align="center">
  <img src="/images/Plot1.png" alt="Plot1 que contiene graficos de las acciones">
</p>


SyntaxError: invalid syntax (2935765058.py, line 30)

In [ ]:
def linear_regression_model(data, colors):
    X = data[['Open', 'High', 'Low', 'Volume', 'Daily_Return']]
    y = data['Next_Day_Price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Mean Squared Error (MSE): {mse}')
    print(f'R-squared (R^2): {r2}')

    # Visualizar los resultados de la regresión
    plot_regression_results(y_test, y_pred, colors)

    return model

In [ ]:
def predict_future_prices(model, data, colors):
    last_day = data.index[-1]
    new_dates = [last_day + BDay(i) for i in range(1, 11)]

    # Calcular el cambio porcentual promedio de los últimos 5 días
    mean_percentage_change = data[['Open', 'High', 'Low', 'Volume', 'Daily_Return']].pct_change().rolling(window=5).mean().iloc[-1]

    # Inicializar un DataFrame para las características futuras
    future_features = pd.DataFrame(index=new_dates, columns=['Open', 'High', 'Low', 'Volume', 'Daily_Return'])
    
    # Aplicar la extrapolación basada en el cambio porcentual promedio
    for i, date in enumerate(future_features.index):
        if i == 0:
            future_features.loc[date] = data[['Open', 'High', 'Low', 'Volume', 'Daily_Return']].iloc[-1]
        else:
            future_features.loc[date] = future_features.iloc[i-1] * (1 + mean_percentage_change)

    future_features = future_features.astype(float)

    # Realizar predicciones con el modelo
    future_predicted_prices = model.predict(future_features)

    # Crear un DataFrame para las fechas y precios pronosticados
    predictions_df = pd.DataFrame({'Date': new_dates, 'Predicted_Price': future_predicted_prices})

    print(predictions_df)